In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import torch
import torch.nn as nn
import os
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.models as models
from collections import defaultdict
from scipy.stats import ttest_rel, wilcoxon


In [30]:
import pandas as pd

# Replace 'your_file.tsv' with the path to your .tsv file
df = pd.read_csv('./data/filtered_results.tsv', sep='\t')

# Display the first few rows
print(df.head())


/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_33507/2273208280.py:4: DtypeWarning: Columns (40,44,46,50,54,58,94,100,104,110,118,124,126,128,130,132,134,136,140,142,146,148,150,152,154,156,158,178,186,192,198,202,214,218,230,236,238,240,244,246,248,250,252,254,256,258,260,262,264,266,270,272,274,276,286,288,290,292,294,296,298,300,302,304,306,308,310,312,314,316,318,320,324,326,332,334,336,338,340,342,344,346,348,352,366,374,376,380,382,384,390,392,394,396,400,406,408,410,412,414,416,418,428,430,442,446,448,456,460,462,466,468,470,486,504,512,516,518,524,526,528,530,540,542,544,546,554,556,558,560,564,578,594,608,610,612,614,620,624,626,628,630,632,634,642,644,646,648,652,654,656,658,660,662,664,666,676,678,680,686,688,694,696,698,700,702,704) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/filtered_results.tsv', sep='\t')


   rowid  ccms_row_id                   Peptidoform  \
0      1            1              .SPLFM+15.995GK.   
1      2            2     .EPQVYTLPPSREEM+15.995TK.   
2      3            3         .AVM+15.995DDFAAFVEK.   
3      4            4  .EFNAETFTFHADIC-33.988TLSEK.   
4      5            5   .M+15.995ADEAGSEADHEGTHSTK.   

               Peptidoform ID      Unmod peptidoform  Total  \
0              SPLFM+15.995GK              .SPLFMGK.  11679   
1     EPQVYTLPPSREEM+15.995TK     .EPQVYTLPPSREEMTK.  11411   
2         AVM+15.995DDFAAFVEK         .AVMDDFAAFVEK.  10949   
3  EFNAETFTFHADIC-33.988TLSEK  .EFNAETFTFHADICTLSEK.  10198   
4   M+15.995ADEAGSEADHEGTHSTK   .MADEAGSEADHEGTHSTK.   9608   

   Total- Unmodified sequence  Peptidoforms- Unmodified sequence  \
0                       11681                                  2   
1                       13133                                 46   
2                       18495                                 81   
3                 

In [31]:
print(df.columns)

Index(['rowid', 'ccms_row_id', 'Peptidoform', 'Peptidoform ID',
       'Unmod peptidoform', 'Total', 'Total- Unmodified sequence',
       'Peptidoforms- Unmodified sequence', 'Proteins', 'Mass',
       ...
       '_dyn_#Patient_M2_healthyMale.Timepoint_2_unmod',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3_unmod', 'id'],
      dtype='object', length=707)


In [32]:
df.shape

(40921, 707)

In [33]:
threshold = 0.9 * df.shape[1]

num_rows_with_90_percent_null = (df.isnull().sum(axis=1) >= threshold).sum()
num_rows_with_90_percent_null

np.int64(26804)

In [34]:
df.shape

(40921, 707)

In [6]:
pattern = r'unmod'
patient_timepoints = []
intensity_cols = [col for col in df.columns if re.search(pattern, col)]
intensity_cols

['Outlier groups- unmod',
 'Outlier group ratio- unmod',
 '_dyn_#Patient_01.Timepoint_1_unmod',
 '_dyn_#Patient_01.Timepoint_2_unmod',
 '_dyn_#Patient_01.Timepoint_3_unmod',
 '_dyn_#Patient_01.Timepoint_4_unmod',
 '_dyn_#Patient_01.Timepoint_5_unmod',
 '_dyn_#Patient_01.Timepoint_6_unmod',
 '_dyn_#Patient_01.Timepoint_7_unmod',
 '_dyn_#Patient_02.Timepoint_1_unmod',
 '_dyn_#Patient_02.Timepoint_2_unmod',
 '_dyn_#Patient_02.Timepoint_3_unmod',
 '_dyn_#Patient_02.Timepoint_4_unmod',
 '_dyn_#Patient_02.Timepoint_5_unmod',
 '_dyn_#Patient_02.Timepoint_6_unmod',
 '_dyn_#Patient_02.Timepoint_7_unmod',
 '_dyn_#Patient_03.Timepoint_1_unmod',
 '_dyn_#Patient_03.Timepoint_2_unmod',
 '_dyn_#Patient_03.Timepoint_3_unmod',
 '_dyn_#Patient_03.Timepoint_4_unmod',
 '_dyn_#Patient_03.Timepoint_5_unmod',
 '_dyn_#Patient_03.Timepoint_6_unmod',
 '_dyn_#Patient_03.Timepoint_7_unmod',
 '_dyn_#Patient_05.Timepoint_1_unmod',
 '_dyn_#Patient_05.Timepoint_2_unmod',
 '_dyn_#Patient_05.Timepoint_3_unmod',
 '_dyn_

In [7]:
df.drop(intensity_cols, axis=1, inplace=True)

In [8]:
print(df.columns)

Index(['rowid', 'ccms_row_id', 'Peptidoform', 'Peptidoform ID',
       'Unmod peptidoform', 'Total', 'Total- Unmodified sequence',
       'Peptidoforms- Unmodified sequence', 'Proteins', 'Mass',
       ...
       '_dyn_#Patient_M1_healthyMale.Timepoint_1',
       '_dyn_#Patient_M1_healthyMale.Timepoint_2',
       '_dyn_#Patient_M1_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_1',
       '_dyn_#Patient_M2_healthyMale.Timepoint_2',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3', 'id'],
      dtype='object', length=369)


In [9]:
df.drop(['rowid', 'ccms_row_id', 'Peptidoform ID', 'Mass'],  axis=1, inplace=True)

In [10]:
df.columns

Index(['Peptidoform', 'Unmod peptidoform', 'Total',
       'Total- Unmodified sequence', 'Peptidoforms- Unmodified sequence',
       'Proteins', 'Charge', 'Num Mods', 'All Mods', 'Is Decoy',
       ...
       '_dyn_#Patient_M1_healthyMale.Timepoint_1',
       '_dyn_#Patient_M1_healthyMale.Timepoint_2',
       '_dyn_#Patient_M1_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_1',
       '_dyn_#Patient_M2_healthyMale.Timepoint_2',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3', 'id'],
      dtype='object', length=365)

In [11]:
df.drop(['Unmod peptidoform', 'Unmod peptidoform', 'Total- Unmodified sequence', 'Peptidoforms- Unmodified sequence', 'Charge', 'Num Mods', 'All Mods', 'Is Decoy'],  axis=1, inplace=True)

In [12]:
for i in df.columns:
    print(i)

Peptidoform
Total
Proteins
Lorikeet input
Orig cluster FDR
Pep Prefix
Annotation
Annotation without position
Known
Num mod frags
PValue
% Explained
Rep cluster task
Rep cluster user
Rep cluster index
Num tasks
Rep spectrum filename
Rep spectrum scan
Outlier groups
Outlier group ratio
Unmod_Peptidoform
_dyn_#Patient_01.Timepoint_1
_dyn_#Patient_01.Timepoint_2
_dyn_#Patient_01.Timepoint_3
_dyn_#Patient_01.Timepoint_4
_dyn_#Patient_01.Timepoint_5
_dyn_#Patient_01.Timepoint_6
_dyn_#Patient_01.Timepoint_7
_dyn_#Patient_02.Timepoint_1
_dyn_#Patient_02.Timepoint_2
_dyn_#Patient_02.Timepoint_3
_dyn_#Patient_02.Timepoint_4
_dyn_#Patient_02.Timepoint_5
_dyn_#Patient_02.Timepoint_6
_dyn_#Patient_02.Timepoint_7
_dyn_#Patient_03.Timepoint_1
_dyn_#Patient_03.Timepoint_2
_dyn_#Patient_03.Timepoint_3
_dyn_#Patient_03.Timepoint_4
_dyn_#Patient_03.Timepoint_5
_dyn_#Patient_03.Timepoint_6
_dyn_#Patient_03.Timepoint_7
_dyn_#Patient_05.Timepoint_1
_dyn_#Patient_05.Timepoint_2
_dyn_#Patient_05.Timepoint_3
_

In [13]:
df.drop(['Total', 'Lorikeet input', 'Orig cluster FDR', 'Pep Prefix',
         'Annotation', 'Annotation without position', 'Known', 
         'Num mod frags', 'PValue',
'Num tasks'],  axis=1, inplace=True)

In [14]:
df.drop([
    "% Explained",
"Rep cluster task",
"Rep cluster user",
"Rep cluster index",
"Rep spectrum filename",
"Rep spectrum scan",
"Outlier groups",
"Outlier group ratio",
"Unmod_Peptidoform"
],  axis=1, inplace=True)

In [15]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_M1_healthyMale.Timepoint_1,_dyn_#Patient_M1_healthyMale.Timepoint_2,_dyn_#Patient_M1_healthyMale.Timepoint_3,_dyn_#Patient_M2_healthyMale.Timepoint_1,_dyn_#Patient_M2_healthyMale.Timepoint_2,_dyn_#Patient_M2_healthyMale.Timepoint_3,_dyn_#Patient_M3_healthyMale.Timepoint_1,_dyn_#Patient_M3_healthyMale.Timepoint_2,_dyn_#Patient_M3_healthyMale.Timepoint_3,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,NaN,"21,822,000","25,470,000",NaN,NaN,NaN,NaN,NaN,NaN,0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,NaN,"68,957,000","188,300,000",NaN,NaN,NaN,NaN,NaN,NaN,2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"287,560,000","617,050","339,430,000",NaN,NaN,NaN,NaN,NaN,NaN,4


In [16]:
df.shape

(40921, 339)

In [17]:
pattern = r'healthy'
patient_timepoints = []
intensity_cols = [col for col in df.columns if re.search(pattern, col)]
print(intensity_cols)
df.drop(intensity_cols, axis=1, inplace=True)

['_dyn_#Patient_F1_healthyFemale.Timepoint_1', '_dyn_#Patient_F1_healthyFemale.Timepoint_2', '_dyn_#Patient_F1_healthyFemale.Timepoint_3', '_dyn_#Patient_F2_healthyFemale.Timepoint_1', '_dyn_#Patient_F2_healthyFemale.Timepoint_2', '_dyn_#Patient_F2_healthyFemale.Timepoint_3', '_dyn_#Patient_F3_healthyFemale.Timepoint_1', '_dyn_#Patient_F3_healthyFemale.Timepoint_2', '_dyn_#Patient_F3_healthyFemale.Timepoint_3', '_dyn_#Patient_M1_healthyMale.Timepoint_1', '_dyn_#Patient_M1_healthyMale.Timepoint_2', '_dyn_#Patient_M1_healthyMale.Timepoint_3', '_dyn_#Patient_M2_healthyMale.Timepoint_1', '_dyn_#Patient_M2_healthyMale.Timepoint_2', '_dyn_#Patient_M2_healthyMale.Timepoint_3', '_dyn_#Patient_M3_healthyMale.Timepoint_1', '_dyn_#Patient_M3_healthyMale.Timepoint_2', '_dyn_#Patient_M3_healthyMale.Timepoint_3']


In [18]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,"3,107,250,000","4,155,225,000","3,817,200,000","6,073,900,000","6,265,175,000","6,710,850,000","6,563,850,000","4,345,225,000","4,934,400,000",0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,"1,834,547,500","3,825,300,000","2,355,000,000","2,594,050,000","2,980,550,000","2,852,100,000","2,884,350,000","1,897,400,000","2,802,800,000",1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,"4,462,570,000","7,384,700,000","8,110,325,000","10,065,025,000","10,515,325,000","9,124,475,000","9,814,875,000","6,985,700,000","9,618,975,000",2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,"3,267,682,500","5,091,600,000","5,787,700,000","8,006,950,000","8,273,600,000","4,958,575,000","6,853,675,000","6,134,675,000","8,260,200,000",3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"674,722,500","1,378,077,500","1,539,647,500","1,638,325,000","2,350,950,000","1,621,445,000","1,828,525,000","1,205,237,500","1,355,452,500",4


In [19]:
grouped = df.groupby('Proteins')


In [20]:
print(len(grouped))

1308


In [21]:
grouped.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,"3,107,250,000","4,155,225,000","3,817,200,000","6,073,900,000","6,265,175,000","6,710,850,000","6,563,850,000","4,345,225,000","4,934,400,000",0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,"1,834,547,500","3,825,300,000","2,355,000,000","2,594,050,000","2,980,550,000","2,852,100,000","2,884,350,000","1,897,400,000","2,802,800,000",1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,"4,462,570,000","7,384,700,000","8,110,325,000","10,065,025,000","10,515,325,000","9,124,475,000","9,814,875,000","6,985,700,000","9,618,975,000",2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,"3,267,682,500","5,091,600,000","5,787,700,000","8,006,950,000","8,273,600,000","4,958,575,000","6,853,675,000","6,134,675,000","8,260,200,000",3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"674,722,500","1,378,077,500","1,539,647,500","1,638,325,000","2,350,950,000","1,621,445,000","1,828,525,000","1,205,237,500","1,355,452,500",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40836,.-11.98NMFWYQQK.,tr|A0A075B6I3|A0A075B6I3_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40836
40841,.+119.037RLEEIMK.,sp|Q14244-2|MAP7_HUMAN;sp|Q14244-3|MAP7_HUMAN;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40841
40855,.+113.91DGAMFFHWRRAAEEGK.,sp|Q9NPF5|DMAP1_HUMAN;tr|Q5TG36|Q5TG36_HUMAN;t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40855
40897,.-1.01HHHHHHHH.,sp|O00257|CBX4_HUMAN;sp|O00555-2|CAC1A_HUMAN;s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40897


In [22]:
grouped.get_group('sp|P01834|IGKC_HUMAN').head()


,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
16,.VYAC+305.068EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"2,289,050,000","1,791,350,000","1,438,220,000","796,622,500","1,897,062,500","1,136,975,000","1,877,952,500","2,878,650,000",...,"2,887,370,000","4,133,000,000","3,874,425,000","4,114,350,000","4,016,050,000","3,517,400,000","4,720,800,000","2,276,950,000","4,766,225,000",16
41,.VYAC-33.988EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"2,174,875,000","1,385,347,500","1,348,430,000","748,765,000","1,715,380,000","2,071,575,000","1,642,750,000","2,555,525,000",...,"2,788,550,000","3,932,850,000","2,735,075,000","4,045,800,000","4,886,500,000","3,119,450,000","4,815,350,000","3,613,475,000","4,590,950,000",41
46,.VYAC+247.10EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46
73,.HKVYAC-33.988EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"114,430,250","60,926,250","55,620,750","58,679,750","132,802,000","165,390,000","76,680,750","182,036,500",...,"186,904,250","416,370,000","184,670,000","606,580,000","811,180,000","155,809,000","520,267,500","317,365,000","422,680,000",73
94,.VDNALQSGNSQE+21.982SVTEQDSK.,sp|P01834|IGKC_HUMAN,"255,890,500","212,915,250","64,215,000","162,854,250","127,326,000","268,752,500","258,960,000","281,722,500",...,"391,183,333","420,325,000","481,067,500","590,280,000","602,140,000","516,605,000","528,785,000","428,280,000","527,342,500",94


In [23]:
grouped["Peptidoform"]

In [24]:
# signif_peptidoformes = [
#     ".EI-14.016VLTQSPGTLSLSPGER.",
#     ".AKLPD+14.016ATPTELAK.",
#     ".LLIYDAS+60.036NR.",
#     ".YVLTQPPSVSVAPGQTAR-28.006.",
#     ".+57.021GPSVFPLAPSSK.",
#     ".EVQLVQSGA+26.016EVK.",
#     ".SGQAPVLVIYEDS+14.016K.",
#     ".RPC+57.021FSA+30.011LEVDETYVPK.",
#     ".YVTSAPM+15.995PEPQAPGR.",
#     ".+43.006C+58.005TESLVNR."
# ]

signif_peptidoformes = [
    ".EI-14.016VLTQSPGTLSLSPGER.",
    ".AKLPD+14.016ATPTELAK.",
    ".YVTSAPM+15.995PEPQAPGR.",
    ".LLIYDAS+60.036NR.",
    ".YVLTQPPSVSVAPGQTAR-28.006.",
    ".+57.021GPSVFPLAPSSK.",
    ".EVQLVQSGA+26.016EVK.",
    ".SGQAPVLVIYEDS+14.016K.",
    ".RPC+57.021FSA+30.011LEVDETYVPK.",
    ".+43.006C+58.005TESLVNR."
]

In [25]:
count = 0
all_significant_proteins = []
for group_name, group_data in grouped["Peptidoform"]:
    # Find which values in the group are in signif_peptidoformes
    matching_values = group_data[group_data.isin(signif_peptidoformes)]
    
    if not matching_values.empty:
        count += 1
        print(f"Group Protein: {group_name}")
        print("Matching Peptidoform(s): ", matching_values)
        print("----------")
        all_significant_proteins.append(group_name)

    # break


Group Protein: sp|A0A075B6K4|LV310_HUMAN
Matching Peptidoform(s):  3052    .SGQAPVLVIYEDS+14.016K.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|A0A0A0MRZ8|KVD11_HUMAN;sp|P04433|KV311_HUMAN
Matching Peptidoform(s):  2348    .LLIYDAS+60.036NR.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|A0A0C4DH38|HV551_HUMAN;sp|A0A0G2JMI3|HV692_HUMAN;sp|A0A0J9YXX1|HV5X1_HUMAN
Matching Peptidoform(s):  1704    .EVQLVQSGA+26.016EVK.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01619|KV301_HUMAN
Matching Peptidoform(s):  116    .EI-14.016VLTQSPGTLSLSPGER.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01857|IGHG1_HUMAN;tr|A0A0A0MS07|A0A0A0MS07_HUMAN;tr|A0A0A0MS08|A0A0A0MS08_HUMAN
Matching Peptidoform(s):  4748    .+57.021GPSVFPLAPSSK.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01871-2|IGHM_HUMAN;sp|P01871|IGHM_HUMAN;tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN
Matching Peptidoform(s):  23    .YVTSAPM+15.995PEPQAPGR.
Name: Peptidofor

In [26]:
all_significant_proteins 

['sp|A0A075B6K4|LV310_HUMAN',
 'sp|A0A0A0MRZ8|KVD11_HUMAN;sp|P04433|KV311_HUMAN',
 'sp|A0A0C4DH38|HV551_HUMAN;sp|A0A0G2JMI3|HV692_HUMAN;sp|A0A0J9YXX1|HV5X1_HUMAN',
 'sp|P01619|KV301_HUMAN',
 'sp|P01857|IGHG1_HUMAN;tr|A0A0A0MS07|A0A0A0MS07_HUMAN;tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
 'sp|P01871-2|IGHM_HUMAN;sp|P01871|IGHM_HUMAN;tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN',
 'sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;sp|P02768|ALBU_HUMAN;tr|A0A087WWT3|A0A087WWT3_HUMAN;tr|A0A0C4DGB6|A0A0C4DGB6_HUMAN;tr|B7WNR0|B7WNR0_HUMAN;tr|C9JKR2|C9JKR2_HUMAN;tr|D6RHD5|D6RHD5_HUMAN;tr|H0YA55|H0YA55_HUMAN',
 'sp|P02774-2|VTDB_HUMAN;sp|P02774-3|VTDB_HUMAN;sp|P02774|VTDB_HUMAN;tr|D6RF35|D6RF35_HUMAN',
 'sp|P80748|LV302_HUMAN']

In [27]:
flat_list = [entry for group in all_significant_proteins for entry in group.split(';')]
flat_list

['sp|A0A075B6K4|LV310_HUMAN',
 'sp|A0A0A0MRZ8|KVD11_HUMAN',
 'sp|P04433|KV311_HUMAN',
 'sp|A0A0C4DH38|HV551_HUMAN',
 'sp|A0A0G2JMI3|HV692_HUMAN',
 'sp|A0A0J9YXX1|HV5X1_HUMAN',
 'sp|P01619|KV301_HUMAN',
 'sp|P01857|IGHG1_HUMAN',
 'tr|A0A0A0MS07|A0A0A0MS07_HUMAN',
 'tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
 'sp|P01871-2|IGHM_HUMAN',
 'sp|P01871|IGHM_HUMAN',
 'tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN',
 'sp|P02768-2|ALBU_HUMAN',
 'sp|P02768-3|ALBU_HUMAN',
 'sp|P02768|ALBU_HUMAN',
 'tr|A0A087WWT3|A0A087WWT3_HUMAN',
 'tr|A0A0C4DGB6|A0A0C4DGB6_HUMAN',
 'tr|B7WNR0|B7WNR0_HUMAN',
 'tr|C9JKR2|C9JKR2_HUMAN',
 'tr|D6RHD5|D6RHD5_HUMAN',
 'tr|H0YA55|H0YA55_HUMAN',
 'sp|P02774-2|VTDB_HUMAN',
 'sp|P02774-3|VTDB_HUMAN',
 'sp|P02774|VTDB_HUMAN',
 'tr|D6RF35|D6RF35_HUMAN',
 'sp|P80748|LV302_HUMAN']

In [28]:
seen = set()
unique_list = []
for item in flat_list:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)

unique_list, len(unique_list)

(['sp|A0A075B6K4|LV310_HUMAN',
  'sp|A0A0A0MRZ8|KVD11_HUMAN',
  'sp|P04433|KV311_HUMAN',
  'sp|A0A0C4DH38|HV551_HUMAN',
  'sp|A0A0G2JMI3|HV692_HUMAN',
  'sp|A0A0J9YXX1|HV5X1_HUMAN',
  'sp|P01619|KV301_HUMAN',
  'sp|P01857|IGHG1_HUMAN',
  'tr|A0A0A0MS07|A0A0A0MS07_HUMAN',
  'tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
  'sp|P01871-2|IGHM_HUMAN',
  'sp|P01871|IGHM_HUMAN',
  'tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN',
  'sp|P02768-2|ALBU_HUMAN',
  'sp|P02768-3|ALBU_HUMAN',
  'sp|P02768|ALBU_HUMAN',
  'tr|A0A087WWT3|A0A087WWT3_HUMAN',
  'tr|A0A0C4DGB6|A0A0C4DGB6_HUMAN',
  'tr|B7WNR0|B7WNR0_HUMAN',
  'tr|C9JKR2|C9JKR2_HUMAN',
  'tr|D6RHD5|D6RHD5_HUMAN',
  'tr|H0YA55|H0YA55_HUMAN',
  'sp|P02774-2|VTDB_HUMAN',
  'sp|P02774-3|VTDB_HUMAN',
  'sp|P02774|VTDB_HUMAN',
  'tr|D6RF35|D6RF35_HUMAN',
  'sp|P80748|LV302_HUMAN'],
 27)

In [117]:
all_significant_proteins = ['sp|P01619|KV301_HUMAN', 'sp|A0A075B6K4|LV310_HUMAN', 'sp|P80748|LV302_HUMAN', 'tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
                            'sp|P01743|HV146_HUMAN', 'sp|A0A0A0MRZ8|KVD11_HUMAN', 'sp|P02774|VTDB_HUMAN', 'tr|D6RHD5|D6RHD5_HUMAN',
                            'sp|P01871|IGHM_HUMAN', 'sp|P01834|IGKC_HUMAN', 'sp|P01721|LV657_HUMAN', 'sp|P02763|A1AG1_HUMAN']
# Parsimony Algorithm

In [118]:
# all_significant_proteins = unique_list

In [119]:
df[df["Proteins"] == 'sp|P01619|KV301_HUMAN'].shape

(177, 321)

In [120]:
df[df["Proteins"] == 'sp|P01871|IGHM_HUMAN'].shape

(0, 321)

In [121]:
df["Proteins"]

0        sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...
1        sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...
2        sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...
3        sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...
4              sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN
                               ...                        
40916    sp|P0C0L4-2|CO4A_HUMAN;sp|P0C0L4|CO4A_HUMAN;sp...
40917          sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN
40918    sp|P02768-3|ALBU_HUMAN;sp|P02768|ALBU_HUMAN;tr...
40919                                 sp|P01834|IGKC_HUMAN
40920    sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...
Name: Proteins, Length: 40921, dtype: object

In [122]:
for i in range(len(all_significant_proteins)):
    filtered_df = df[all_significant_proteins[i] == df["Proteins"]]
    # file_name = f"{all_significant_proteins[i]}.csv"
    file_path = os.path.join(f"./data/filetered_proteins/ModifiedFiles/{i+1}.csv")
    filtered_df.to_csv(file_path, index=True)
    print(f"Saved: {file_path}")
    print(filtered_df.shape)

Saved: ./data/filetered_proteins/ModifiedFiles/1.csv
(177, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/2.csv
(35, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/3.csv
(143, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/4.csv
(0, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/5.csv
(5, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/6.csv
(0, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/7.csv
(0, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/8.csv
(6, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/9.csv
(0, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/10.csv
(1288, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/11.csv
(62, 321)
Saved: ./data/filetered_proteins/ModifiedFiles/12.csv
(188, 321)


In [123]:
# for i, protein in enumerate(all_significant_proteins, start=1):
#     filtered_df = df[df["Proteins"].str.contains(protein, na=False)]
    
#     file_path = os.path.join(f"./data/filetered_proteins/{i}.csv")
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     filtered_df.to_csv(file_path, index=True)
    
#     print(filtered_df.shape)



In [90]:
df = pd.read_csv("./data/filetered_proteins/ModifiedFiles/6.csv")
df.head()

,Unnamed: 0,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id


In [63]:
df.head(), df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Columns: 322 entries, Unnamed: 0 to id
dtypes: float64(56), int64(2), object(264)
memory usage: 88.2+ KB


(   Unnamed: 0                      Peptidoform                   Proteins  \
 0        3052          .SGQAPVLVIYEDS+14.016K.  sp|A0A075B6K4|LV310_HUMAN   
 1        6560          .SGQAPVLVIYEDS+27.011K.  sp|A0A075B6K4|LV310_HUMAN   
 2        7078          .SGQAPVLVIY-15.995EDSK.  sp|A0A075B6K4|LV310_HUMAN   
 3        7236   .SGQAPVLVIY-75.10EDSKRPSGIPER.  sp|A0A075B6K4|LV310_HUMAN   
 4        7583  .SGQAPVLVIYEDSK+28.006RPSGIPER.  sp|A0A075B6K4|LV310_HUMAN   
 
   _dyn_#Patient_01.Timepoint_1  _dyn_#Patient_01.Timepoint_2  \
 0                          NaN                           NaN   
 1                          NaN                           NaN   
 2                          NaN                           NaN   
 3                          NaN                           NaN   
 4                          NaN                           NaN   
 
   _dyn_#Patient_01.Timepoint_3 _dyn_#Patient_01.Timepoint_4  \
 0                          NaN                          NaN   
 1        

In [64]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [70]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38405000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052


In [66]:
threshold = 0.6  # 90% threshold
max_na = df.shape[1] * threshold

df = df[~((df.isna() | (df == '')).sum(axis=1) > max_na)].reset_index(drop=True)


In [67]:
df.shape

(1, 321)

In [68]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"38,405,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052


In [69]:
object_cols = df.select_dtypes(include='object').columns

# Convert comma-separated numeric strings to actual numbers
for col in object_cols:
    if col not in ['Peptidoform', 'Proteins']:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

df.head()


,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38405000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052


In [36]:
timepoints = [col for col in df.columns if any(tp in col.lower() for tp in ['_1', '_2', '_3', '_4', '_5', '_6', '_7'])]

for index, row in df.iterrows():
    for col in timepoints:
        if pd.isna(row[col]):
            if df[col].dtype in ['float64', 'int64']:
                median_val = row[timepoints].median(skipna=True)
                df.at[index, col] = median_val
            elif df[col].dtype == 'object':
                mode_val = row[timepoints].mode(dropna=True)
                if not mode_val.empty:
                    df.at[index, col] = mode_val[0]



In [37]:
df.columns

Index(['Peptidoform', 'Proteins', '_dyn_#Patient_01.Timepoint_1',
       '_dyn_#Patient_01.Timepoint_2', '_dyn_#Patient_01.Timepoint_3',
       '_dyn_#Patient_01.Timepoint_4', '_dyn_#Patient_01.Timepoint_5',
       '_dyn_#Patient_01.Timepoint_6', '_dyn_#Patient_01.Timepoint_7',
       '_dyn_#Patient_02.Timepoint_1',
       ...
       '_dyn_#Patient_57.Timepoint_6', '_dyn_#Patient_57.Timepoint_7',
       '_dyn_#Patient_58.Timepoint_1', '_dyn_#Patient_58.Timepoint_2',
       '_dyn_#Patient_58.Timepoint_3', '_dyn_#Patient_58.Timepoint_4',
       '_dyn_#Patient_58.Timepoint_5', '_dyn_#Patient_58.Timepoint_6',
       '_dyn_#Patient_58.Timepoint_7', 'id'],
      dtype='object', length=321)

In [38]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,38405000,...,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,3052


In [39]:
patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]
df[patient_cols] = np.log2(df[patient_cols] + 1)

In [40]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.194791,...,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,3052


In [41]:
df.columns

Index(['Peptidoform', 'Proteins', '_dyn_#Patient_01.Timepoint_1',
       '_dyn_#Patient_01.Timepoint_2', '_dyn_#Patient_01.Timepoint_3',
       '_dyn_#Patient_01.Timepoint_4', '_dyn_#Patient_01.Timepoint_5',
       '_dyn_#Patient_01.Timepoint_6', '_dyn_#Patient_01.Timepoint_7',
       '_dyn_#Patient_02.Timepoint_1',
       ...
       '_dyn_#Patient_57.Timepoint_6', '_dyn_#Patient_57.Timepoint_7',
       '_dyn_#Patient_58.Timepoint_1', '_dyn_#Patient_58.Timepoint_2',
       '_dyn_#Patient_58.Timepoint_3', '_dyn_#Patient_58.Timepoint_4',
       '_dyn_#Patient_58.Timepoint_5', '_dyn_#Patient_58.Timepoint_6',
       '_dyn_#Patient_58.Timepoint_7', 'id'],
      dtype='object', length=321)

In [124]:

# patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]

def normalize_row(row):
    min_val = row.min()
    max_val = row.max()
    if min_val == max_val:
        return pd.Series([0] * len(row), index=row.index)  # or keep it as-is
    return 2 * ((row - min_val) / (max_val - min_val)) - 1

# df[patient_cols] = df[patient_cols].apply(normalize_row, axis=1)


In [43]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,0.04398,0.04398,0.04398,0.04398,0.04398,0.04398,0.04398,-0.434723,...,0.04398,0.04398,0.04398,0.04398,0.04398,0.04398,0.04398,0.04398,0.04398,3052


In [44]:

patient_timepoints = defaultdict(list)
for col in patient_cols:
    parts = col.split('.Timepoint_')
    if len(parts) == 2:
        patient_id = parts[0]
        patient_timepoints[patient_id].append(col)

differences = pd.DataFrame()
for patient_id, cols in patient_timepoints.items():
    sorted_cols = sorted(cols, key=lambda x: int(x.split('.Timepoint_')[1]))
    first_col = sorted_cols[0]
    last_col = sorted_cols[-1]
    diff_col_name = f"{patient_id}_range"
    differences[diff_col_name] = df[last_col] - df[first_col]


In [45]:
# differences['Peptidoform'] = df['Peptidoform']
# differences['Proteins'] = df['Proteins']
print(differences.shape)
differences.head()

(1, 52)


,_dyn_#Patient_01_range,_dyn_#Patient_02_range,_dyn_#Patient_03_range,_dyn_#Patient_05_range,_dyn_#Patient_06_range,_dyn_#Patient_07_range,_dyn_#Patient_08_range,_dyn_#Patient_09_range,_dyn_#Patient_10_range,_dyn_#Patient_11_range,...,_dyn_#Patient_47_range,_dyn_#Patient_48_range,_dyn_#Patient_49_range,_dyn_#Patient_50_range,_dyn_#Patient_51_range,_dyn_#Patient_52_range,_dyn_#Patient_53_range,_dyn_#Patient_54_range,_dyn_#Patient_57_range,_dyn_#Patient_58_range
0,0.0,-0.127905,0.0,0.0,0.0,0.0,-0.879701,0.0,0.0,0.697143,...,0.324152,0.0,0.0,0.0,-0.036706,0.0,0.0,0.0,0.0,0.0


In [46]:
values = differences.mean(axis=1)
values

0   -0.015672
dtype: float64

In [287]:
file_path = os.path.join(f"./data/filetered_proteins/{+1}.csv")
filtered_df.to_csv(file_path, index=True)

In [125]:
rootDir = './data/filetered_proteins/ModifiedFiles/'
for filename in os.listdir(rootDir):
    df = pd.read_csv(rootDir+filename)
    # print(df.head())
    df.drop('Unnamed: 0', axis=1, inplace=True)
    print(f"Data Shape for {filename} is {df.shape}")
    threshold = 0.9  # 90% threshold
    max_na = df.shape[1] * threshold

    df = df[~((df.isna() | (df == '')).sum(axis=1) > max_na)].reset_index(drop=True)
    print(f"Data Shape for {filename} after dropping 90% data {df.shape}")
    object_cols = df.select_dtypes(include='object').columns

    # Convert comma-separated numeric strings to actual numbers
    for col in object_cols:
        if col not in ['Peptidoform', 'Proteins']:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')


    timepoints = [col for col in df.columns if any(tp in col.lower() for tp in ['_1', '_2', '_3', '_4', '_5', '_6', '_7'])]


    for index, row in df.iterrows():
        for col in timepoints:
            if pd.isna(row[col]):
                if df[col].dtype in ['float64', 'int64']:
                    median_val = row[timepoints].median(skipna=True)
                    df.at[index, col] = median_val
                elif df[col].dtype == 'object':
                    mode_val = row[timepoints].mode(dropna=True)
                    if not mode_val.empty:
                        df.at[index, col] = mode_val[0]

    patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]
    df[patient_cols] = np.log2(df[patient_cols] + 1)
    df[patient_cols] = df[patient_cols].apply(normalize_row, axis=1)

    patient_timepoints = defaultdict(list)
    for col in patient_cols:
        parts = col.split('.Timepoint_')
        if len(parts) == 2:
            patient_id = parts[0]
            patient_timepoints[patient_id].append(col)

    initial_timeFrames = []
    final_timeFrames = []
    differences = pd.DataFrame()
    for patient_id, cols in patient_timepoints.items():
        sorted_cols = sorted(cols, key=lambda x: int(x.split('.Timepoint_')[1]))
        first_col = sorted_cols[0]
        last_col = sorted_cols[-1]
        diff_col_name = f"{patient_id}_range"
        change = df[last_col] - df[first_col]
        differences[diff_col_name] = (change/df[first_col]) * 100
        initial_timeFrames.append(df[first_col])
        final_timeFrames.append(df[last_col])
    
    print(len(initial_timeFrames), len(final_timeFrames))
    Tfirst_values = np.array(initial_timeFrames, dtype = float)  # Columns 1 to 58 (P1_T1 to P58_T1)
    Tlast_values = np.array(final_timeFrames, dtype = float) # Columns 59 to 116 (P1_T7 to P58_T7)

    t_stat, p_ttest = ttest_rel(Tlast_values, Tfirst_values)
    w_stat, p_wilcoxon = wilcoxon(Tlast_values, Tfirst_values)

    differences['paired-t'] = t_stat
    differences['paired-p'] = p_ttest
    differences['WilCoxnRank-w'] = w_stat
    differences['WilCoxnRank-p'] = p_wilcoxon

    print(t_stat.shape, p_ttest.shape, w_stat.shape, p_wilcoxon.shape)

    row_means = differences.mean(axis=1)
    differences["avg"] = row_means
    differences['Peptidoform'] = df['Peptidoform']
    differences['Proteins'] = df['Proteins']

    file_path = os.path.join(f"./data/log_data/ModifiedFiles/{filename}")
    differences.to_csv(file_path, index=True) 
        


Data Shape for 6.csv is (0, 321)
Data Shape for 6.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 7.csv is (0, 321)
Data Shape for 7.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 5.csv is (5, 321)
Data Shape for 5.csv after dropping 90% data (1, 321)
52 52
(1,) (1,) (1,) (1,)
Data Shape for 4.csv is (0, 321)
Data Shape for 4.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 1.csv is (177, 321)
Data Shape for 1.csv after dropping 90% data (41, 321)
52 52
(41,) (41,) (41,) (41,)
Data Shape for 3.csv is (143, 321)
Data Shape for 3.csv after dropping 90% data (23, 321)
52 52
(23,) (23,) (23,) (23,)
Data Shape for 2.csv is (35, 321)
Data Shape for 2.csv after dropping 90% data (4, 321)
52 52
(4,) (4,) (4,) (4,)
Data Shape for 10.csv is (1288, 321)
Data Shape for 10.csv after dropping 90% data (139, 321)
52 52
(139,) (139,) (139,) (139,)
Data Shape for 11.csv is (62, 321)
Data Shape for 11.csv after dro